###Importamos las librerias que nos seran utiles

In [372]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import seaborn as sns

#Importamos los datos

In [373]:
sharks = pd.read_csv('/home/julian/Cursos/Ironhack/Proyecto1/shark-data-cleaning/attacks.csv', encoding="ISO-8859-1")

In [374]:
sdf = pd.DataFrame(sharks)

##Como podemos observar, la mayoria de los datos son no numericos.

In [375]:
sdf.dtypes

Case Number                object
Date                       object
Year                      float64
Type                       object
Country                    object
Area                       object
Location                   object
Activity                   object
Name                       object
Sex                        object
Age                        object
Injury                     object
Fatal (Y/N)                object
Time                       object
Species                    object
Investigator or Source     object
pdf                        object
href formula               object
href                       object
Case Number.1              object
Case Number.2              object
original order            float64
Unnamed: 22                object
Unnamed: 23                object
dtype: object

### LIMPIEZA HIPOTESIS I

##Vamos a eliminar las columnas que no son relevantes para el analisis de la primer hipotesis

In [376]:
def drop():
    relevant = ['Case Number', 'Type', 'Case Number.1', 'Case Number.2']
    for i in sdf.columns:
        if i not in relevant:
            sdf.drop(columns=[i], inplace=True)

In [377]:
drop()

#Las columnas 'Case Number', 'Case Number.1' y 'Case Number.2' son relevantes para el analisis, sin embargo, vamos a chequear en que medida los valores de las columnas se parecen para prescindir de alguna de las tres ##DEFINIR FUNCION

In [378]:
"""
def compare(): 
        compare1 = np.where(sdf['Case Number'] == sdf['Case Number.1'], True, False)
        sdf['equal1'] = compare1
        compare2 = np.where(sdf['Case Number'] == sdf['Case Number.2'], True, False)
        sdf['equal2'] = compare2
        compare3 = np.where(sdf['Case Number'] == sdf['Case Number.2'], True, False)
        sdf['equal3'] = compare3
        return print(sdf['equal1'].value_counts()); print(sdf['equal2'].value_counts()); print(sdf['equal3'].value_counts())
"""
            

"\ndef compare(): \n        compare1 = np.where(sdf['Case Number'] == sdf['Case Number.1'], True, False)\n        sdf['equal1'] = compare1\n        compare2 = np.where(sdf['Case Number'] == sdf['Case Number.2'], True, False)\n        sdf['equal2'] = compare2\n        compare3 = np.where(sdf['Case Number'] == sdf['Case Number.2'], True, False)\n        sdf['equal3'] = compare3\n        return print(sdf['equal1'].value_counts()); print(sdf['equal2'].value_counts()); print(sdf['equal3'].value_counts())\n"

In [379]:
compare = np.where(sdf['Case Number.1'] == sdf['Case Number.2'], True, False)

In [380]:
sdf['equal'] = compare

In [381]:
sdf['equal'].value_counts() 

False    19441
True      6282
Name: equal, dtype: int64

In [382]:
sdf.isnull().sum()

Case Number      17021
Type             19425
Case Number.1    19421
Case Number.2    19421
equal                0
dtype: int64

##SI COMPARAMOS LAS TRES COL ENTRE SI, EN DONDE MENOS DIF DE VAL HAY ES ENTRE CAS NBR Y CAS NBR.2, AUNQUE LA DI ES MINIMA. SIN EMBARGO DE LAS TRES COL LA QUE MENOS CANT DE NULL TIENE ES CASE NBR, NOS QUEDAMOS CON ELLA##DEJAMOS SOLO LAS COLUMNAS CASE NR Y TYPE

In [383]:
sdf.drop(columns=['Case Number.1', 'Case Number.2', 'equal'], inplace=True)

##VAMOS A ELIMINAR LOS VALORES QUE NO SEAN FECHA, YA QUE NOS INTERESA VER LA ESTACIONALIDAD. Empezamos por los valores NaN, y luego añadiremos una columna solo con los valores que contengan informacion sobre el mes, ya que nos interesa solo la indfo mensual

In [384]:
sdf.dropna(inplace=True)

In [385]:
def filtracion(x):
    months = {'01':'Jan','02':'Feb','03':'Mar','04':'Apr','05':'May','06':'Jun','07':'Jul','08':'Aug','09':'Sep','10':'Oct','11':'Nov','12':'Dec'}
    m = re.findall('\d{4}.\d{2}.\d{2}', x)
    if m and m[0][5:7] != '00':
        for k, v in months.items():
            if k == m[0][5:7]:
                return v

In [386]:
sdf['Months'] = sdf['Case Number'].apply(filtracion)

##Borramos los valores nulos para los datos que no tenian registro mensual

In [387]:
sdf.dropna(inplace=True)

##exportamos el data frame a un csv para realizar las visualizaciones

In [388]:
sdf.to_csv('sdf',index=False)

##aHORA CONSTRUIMOS UN DATAFRAME CUYAS COLUMNAS CONTENDRAN LAS FRECUENCIAS MENSUALES DE ATAQUES Y LOS MESES CORRESPNDIENTES consideramos todos los tipos  de ataque.

In [389]:
ataques_mensuales = pd.DataFrame(sdf['Months'].value_counts()).reset_index()

##RENOMBRAMOS LAS COLUMNAS PARA EMPROLIJAR

In [390]:
c = {'Months': 'Monthly Attacks', 'index':'Month'}
ataques_mensuales = ataques_mensuales.rename(columns=c)

In [391]:
ataques_mensuales = ataques_mensuales.reindex([3, 11, 8, 5, 10, 4, 0, 1, 2, 7, 9, 6])

In [392]:
display(ataques_mensuales)

,Month,Monthly Attacks
3,Jan,520
11,Feb,382
8,Mar,409
5,Apr,457
10,May,391
4,Jun,499
0,Jul,668
1,Aug,599
2,Sep,552
7,Oct,441


##Exportamos el data frame de ataques mensuales para utilizarlo en las visualizaciones

In [393]:
ataques_mensuales.to_csv('ataques_mensuales',index=False)

### LIMPIEZA HIPOTESIS II

In [420]:
sdf2= pd.DataFrame(sharks)

### vamos a quedarnos solo con las columnas que nos interesan para analizar la H2, 'Species ' , 'Injury' y 'Fatal'

In [421]:
def drop2():
    """
    Por cada elemento perteneciente al index de columnas, si este no pertenece a la lista de columnas
    relevantes, lo elimina del data frame
    """
    relevant = ['Injury', 'Fatal (Y/N)', 'Species ']
    for i in sdf2.columns:
        if i not in relevant:
            sdf2.drop(columns=[i], inplace=True)

In [422]:
drop2()

### vamos a limpiar las col 'Injury' y 'Fatal'

##Si bien ambas columnas poseen datos sobre la letalidad, la columna 'Sepecies' es la que mas datos nulos posee. Comenzaremos limpiando esa columna

In [423]:
sdf2['Fatal (Y/N)'].isna().value_counts()

True     19960
False     5763
Name: Fatal (Y/N), dtype: int64

In [424]:
sdf2['Injury'].isna().value_counts()

True     19449
False     6274
Name: Injury, dtype: int64

In [425]:
sdf2['Species '].isna().value_counts()

True     22259
False     3464
Name: Species , dtype: int64

In [426]:
sdf2.dropna(inplace=True)

In [427]:
sdf2.shape

(2944, 3)

In [428]:
def filtracion_especies(x):
    m = re.findall('\w+. shark', x)
    if m:
        return (m[0].lower())

In [429]:
sdf2['Identified Specie'] = sdf2['Species '].apply(filtracion_especies)

### Vamos a armar dos data frames, uno con las muertes y otro que considere todos los ataques

### Ataques x especie

In [430]:
ataques = pd.DataFrame(sdf2['Identified Specie'].value_counts()).reset_index()

In [431]:
d = {'Identified Specie': 'Ataques', 'index':'Especie'}

In [432]:
ataques = ataques.rename(columns=d)

In [433]:
ataques.shape

(127, 2)

#### Como hay muchas categorias de tiburones con tan solo un registro, vamos a considerar solo las que tengan al menos 10 registros de ataque.

In [434]:
ataques = ataques[ataques['Ataques'] > 5]

In [435]:
ataques.shape

(45, 2)

In [418]:
ataques.to_csv('Ataques-especies',index=False)

### Letalidad por especie

#### Como la columna Fatal posee mas valores no nulos que Injury, potencialmente podemos obtener mas info sobre letalidad, vamos a prescindir de Injury. Vamos a quedarnos con los valores en donde hay un 'Y' en Fatal, en la col letalidad

In [436]:
sdf2.drop(columns=['Injury'], inplace=True)

In [437]:
letal = pd.DataFrame(sdf2[sdf2['Fatal (Y/N)'] == 'Y'])

In [438]:
letal2 = pd.DataFrame(letal['Identified Specie'].value_counts()).reset_index()

In [439]:
d = {'Identified Specie': 'Cantidad de Muertes', 'index':'Especie'}
letal2 = letal2.rename(columns=d)

#### Como podemos ver, a partir de la septima fila, los registros son muy, bajos, vamos a recortar el data frame para quedarnos con las especies que mas registros tienen

In [440]:
letal2.drop(range(8, 41), axis=0, inplace=True)

In [441]:
letal2.to_csv('especies-letalidad',index=False)

## HIPOTESIS III